In [1]:
pip install -r ../../requirements.txt

You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [68]:
import ezkl
import torch
from torch import nn
import json
import os
import time
import scipy
import numpy as np
import matplotlib.pyplot as plt
import statistics
import math

In [69]:
%run -i ../../core.py

In [70]:
# init path
os.makedirs(os.path.dirname('shared/'), exist_ok=True)
os.makedirs(os.path.dirname('prover/'), exist_ok=True)
verifier_model_path = os.path.join('shared/verifier.onnx')
prover_model_path = os.path.join('prover/prover.onnx')
verifier_compiled_model_path = os.path.join('shared/verifier.compiled')
prover_compiled_model_path = os.path.join('prover/prover.compiled')
pk_path = os.path.join('shared/test.pk')
vk_path = os.path.join('shared/test.vk')
proof_path = os.path.join('shared/test.pf')
settings_path = os.path.join('shared/settings.json')
srs_path = os.path.join('shared/kzg.srs')
witness_path = os.path.join('prover/witness.json')
# this is private to prover since it contains actual data
comb_data_path = os.path.join('prover/comb_data.json')

=======================  ZK-STATS FLOW =======================

In [84]:
def mode_within(data_array, percent):
    max_sum_freq = 0
    mode = data_array[0]

    for check_val in set(data_array):
        sum_freq = sum(1 for ele in data_array if abs(ele - check_val) <= abs(percent * check_val / 100))

        if sum_freq > max_sum_freq:
            mode = check_val
            max_sum_freq = sum_freq

    return mode

In [85]:
data_path = os.path.join('data.json')
dummy_data_path = os.path.join('shared/dummy_data.json')

f_raw_input = open(data_path, "r")
data = json.loads(f_raw_input.read())["input_data"][0]
data_tensor = torch.reshape(torch.tensor(data),(1, len(data), 1))


dummy_data = np.round(np.random.uniform(1,10,len(data)),1)
json.dump({"input_data":[dummy_data.tolist()]}, open(dummy_data_path, 'w'))

dummy_data_tensor = torch.reshape(torch.tensor(dummy_data), (1, len(dummy_data),1 ))
dummy_theory_output = torch.tensor(mode_within(dummy_data,1))




In [86]:
print("dummy output: ", dummy_theory_output)
# Verifier/ data consumer side: send desired calculation
class verifier_model(nn.Module):
    def __init__(self):
        super(verifier_model, self).__init__()
        # w represents mean in this case
        self.w = nn.Parameter(data = dummy_theory_output, requires_grad = False)
    def forward(self,X):
        count_equal = torch.sum((torch.abs(X-self.w)<=torch.abs(0.01*self.w)).double())
        result = torch.zeros(X.size()[1])
        index_array = torch.tensor(range(X.size()[1]))

        for index in index_array:
            ele = X[0][index][0]
            if (torch.sum((torch.abs(X-ele)<=torch.abs(0.01*ele)).double())<=count_equal):
                check_bool = torch.tensor(True)
            else:
                check_bool = torch.tensor(False)
            result[index] = check_bool

        return (torch.sum(result) == X.size()[1], self.w)
    
verifier_define_calculation(verifier_model, verifier_model_path, [dummy_data_path])

dummy output:  tensor(8.5000, dtype=torch.float64)


/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_41683/297875866.py:11: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  index_array = torch.tensor(range(X.size()[1]))
/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_41683/297875866.py:13: TracerWarning: Iterating over a tensor might cause the trace to be incorrect. Passing a tensor of different shape won't change the number of iterations executed (and might lead to errors or silently give incorrect results).
  for index in index_array:
/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_41683/297875866.py:15: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this 

In [87]:
# prover calculates settings, send to verifier

theory_output = torch.tensor(mode_within(data,1))

print("Theory_output: ", theory_output)
class prover_model(nn.Module):
    def __init__(self):
        super(prover_model, self).__init__()
        # w represents mean in this case
        self.w = nn.Parameter(data = theory_output, requires_grad = False)

    def forward(self,X):
        count_equal = torch.sum((torch.abs(X-self.w)<=torch.abs(0.01*self.w)).double())
        result = torch.zeros(X.size()[1])
        index_array = torch.tensor(range(X.size()[1]))

        for index in index_array:
            ele = X[0][index][0]
            if (torch.sum((torch.abs(X-ele)<=torch.abs(0.01*ele)).double())<=count_equal):
                check_bool = torch.tensor(True)
            else:
                check_bool = torch.tensor(False)
            result[index] = check_bool
        return (torch.sum(result) == X.size()[1], self.w)

prover_gen_settings([data_path], comb_data_path, prover_model,prover_model_path, "default", "resources", settings_path)

Theory_output:  tensor(77.)


/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_41683/3133542595.py:15: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  index_array = torch.tensor(range(X.size()[1]))
/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_41683/3133542595.py:17: TracerWarning: Iterating over a tensor might cause the trace to be incorrect. Passing a tensor of different shape won't change the number of iterations executed (and might lead to errors or silently give incorrect results).
  for index in index_array:
/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_41683/3133542595.py:19: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so th

==== Generate & Calibrate Setting ====
scale:  default
setting:  {"run_args":{"tolerance":{"val":0.0,"scale":1.0},"input_scale":7,"param_scale":7,"scale_rebase_multiplier":10,"lookup_range":[0,0],"logrows":14,"num_inner_cols":1,"variables":[["batch_size",1]],"input_visibility":{"Hashed":{"hash_is_public":true,"outlets":[]}},"output_visibility":"Public","param_visibility":"Private"},"num_rows":14432,"total_assignments":300,"total_const_size":0,"model_instance_shapes":[[1],[1]],"model_output_scales":[0,7],"model_input_scales":[7],"module_sizes":{"kzg":[],"poseidon":[14432,[1]],"elgamal":[0,[0]]},"required_lookups":[],"check_mode":"UNSAFE","version":"5.0.8","num_blinding_factors":null}


In [88]:
# Here verifier & prover can concurrently call setup since all params are public to get pk. 
# Here write as verifier function to emphasize that verifier must calculate its own vk to be sure
verifier_setup(verifier_model_path, verifier_compiled_model_path, settings_path, srs_path,vk_path, pk_path )

print("=======================================")
# Prover generates proof
print("Theory output: ", theory_output)
prover_gen_proof(prover_model_path, comb_data_path, witness_path, prover_compiled_model_path, settings_path, proof_path, pk_path, srs_path)

spawning module 0


==== setting up ezkl ====


spawning module 2
spawning module 0
spawning module 2


Time setup: 1.201237440109253 seconds
Theory output:  tensor(77.)
==== Generating Witness ====
witness boolean:  1.0
witness result 1 : 77.0
==== Generating Proof ====


spawning module 0
spawning module 2


proof:  {'instances': [[[12448964500226922122, 1569304359747940647, 1012944616373047521, 396517790474193570], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [8814853846036591701, 18168188371856170531, 4258635471548586724, 2506957641253187322]]], 'proof': '28d1b2cb37dacc65ece35d2ea30e286c3ffa22fa39c7c997476cb938891209232cc257c4bbbe8bb38981d9d9e22378e4d2b28c932e70a79aa8a0a121b07993eb12976718e9db247008f18254c3cfe43b328e13bb07422e59fad8efaf2db34e180359f4378003bacc0b3620346e67f8960b05002bef60fff26840925152a442c8105c0d2ca8431c937a4fe6c6617ef545d325b9dd0ab30df6b403a87df234e962023f9d3b99c5616b71e59eab22c8f9595c254a441b9b720e4d3c483057b7a1890e3da74d913142a85cc5a299ec30712fb48f82718213f63dff41d97f000e02a90a004a1f3f430432b0aa5145631d6036f6efc2498f73729717325d1ac3828fae048bbd7378dece6fd9629eb10f6831c9854b8ffd5709d19e14a72835c81026bc194f2f7c17506f2ed620de8a4fd69c8d5396b4dc075c74208987503c62ef41c614198585c930bf8ebc198b8bfcff2ecc3bb8a94842eb2651c1c4bbf237b60b62

In [89]:
# Verifier verifies
verifier_verify(proof_path, settings_path, vk_path, srs_path)

num_inputs:  1
prf instances:  [[[12448964500226922122, 1569304359747940647, 1012944616373047521, 396517790474193570], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [8814853846036591701, 18168188371856170531, 4258635471548586724, 2506957641253187322]]]
proof boolean:  1.0
proof result 1 : 77.0
verified
